# Capstone Project - Which Shoe is the Best for You?

General Assembly passion project. Scrape or obtain data from resources online to develop a dataset to perform cleaning, EDA, and analysis on. Try to predict a model on common themes like: 

- Price
- If item is in category A or B
- Cluster and create groups
- Recommender

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import urllib
from bs4 import BeautifulSoup
import requests
from time import sleep, strftime

**Testing one website and finding appropriate keys**

Then run again on page 2.

In [2]:
result = requests.get('https://stockx.com/api/browse?page=1&category=152')
json_res = result.json()

# print json_res['Products'][0]['shortDescription']
# print json_res['Products'][0]['retailPrice']

In [3]:
result = requests.get('https://stockx.com/api/browse?page=2&category=152')
json_res = result.json()

print json_res['Products'][0]['shortDescription']
print json_res['Products'][0]['retailPrice']

Air-Jordan-1-Retro-Top-3
160


In [4]:
json_res['Products'][0]['market']

{u'absChangePercentage': 0.143243,
 u'annualHigh': 548,
 u'annualLow': 160,
 u'averageDeadstockPrice': 318,
 u'averageDeadstockPriceRank': 48,
 u'changePercentage': -0.143243,
 u'changeValue': -53,
 u'createdAt': u'2016-10-07T19:00:03+00:00',
 u'deadstockRangeHigh': 341,
 u'deadstockRangeLow': 293,
 u'deadstockSold': 3630,
 u'deadstockSoldRank': 15,
 u'highestBid': 320,
 u'lastHighestBidTime': 1497924191,
 u'lastLowestAskTime': 1498068907,
 u'lastSale': 317,
 u'lastSaleDate': u'2017-06-21T23:19:11+00:00',
 u'lowestAsk': 180,
 u'pricePremium': 0.981,
 u'pricePremiumRank': 43,
 u'productId': 0,
 u'productUuid': u'5dd63eea-6977-4d47-842c-d1c00f08a247',
 u'salesLast72Hours': 43,
 u'salesLastPeriod': 0,
 u'salesThisPeriod': 43,
 u'skuUuid': None,
 u'updatedAt': 1498101634,
 u'volatility': 0.074643}

### Functions to run for loop to scrape website

First function scrapes the web for an amount of pages (default 50). It will save the raw data as a csv as well. The second function will then clean the dataframe by taking out the unused columns.

In [5]:
def shoe_scraper(pages=50):
    '''Returns one dataframe of all results. 
    And will save into a new file.'''
    
    # Have to run requests first to get appropriate column names
    req = requests.get('https://stockx.com/api/browse?page=1&category=152')
    json_req = req.json()
    df = pd.DataFrame([], columns=json_req['Products'][0].keys())

    for i in range(1,pages):
        try:
            html = 'https://stockx.com/api/browse?page=' + str(int(i)) + '&category=152'
            result = requests.get(html)
            json_res = result.json()
            df = pd.concat([df, pd.DataFrame(json_res['Products'])])
            sleep(0.5)
        except:
            break
    
    # Drop row duplicates
    df.drop_duplicates(['shortDescription', 'urlKey'], inplace=True) 
    
    # Function to save as csv file under today's day as raw (before dropping)
    def csv_maker(df):
        filename = 'StockX_' + strftime("%m%d%H")
        df.to_csv(path_or_buf='C:\\Users\\Chris\\Desktop\\dsi-atl-3\\project\\Capstone\\datasets\\' + filename, encoding='utf-8')
    
    csv_maker(df)
    
    return df

In [6]:
def clean_df(busy_dataframe):
    '''Dropping columns that hold little to no information.
    Then reset the index since we are getting repeated indices'''
    
    # Market DataFrame
    market_df = pd.DataFrame([row for row in busy_dataframe['market']])
    
    
    # Drop unnecessary columns
    cleaner_dataframe = busy_dataframe.join(market_df)
    
    cleanest_dataframe = cleaner_dataframe.drop(['breadcrumbs', 'childId', 'countryOfManufacture', 'type', 
        'uuid', 'dataType', 'doppelgangers', 'condition', 'description', 'hidden', 'ipoDate', 'productCategory', 
        'shoeSize', 'urlKey', 'charityCondition', 'releaseTime', 'shortDescription', 'media', '_highlightResult', 
        'market', '_tags', 'id', 'objectID', 'lastHighestBidTime', 'lastLowestAskTime', 'styleId', 'productId',
        'productUuid', 'skuUuid', 'updatedAt', 'title', 'traits', 'tickerSymbol', 'salesLastPeriod'], axis=1)
    
    # Remember title = shoe + name

    # Reset the index, since we are getting repeated indices
    cleanest_dataframe.reset_index(drop=True, inplace=True)
    return cleanest_dataframe

In [9]:
scraped_shoe = shoe_scraper()